# Init

In [1]:
using LinearAlgebra
using NearestNeighbors
using StaticArrays
include("utils.jl"   )
include("kernels.jl" );

query_value_w_gauss

In [2]:
_DIM_X    = 2
_DIM_A    = 2
_X_DOMAIN = [-2.0 +2.0; -2.0 +2.0]
_A_DOMAIN = [-0.5 +0.5; -0.5 +0.5]
_Q_DOMAIN = [_X_DOMAIN; _A_DOMAIN]
_LEAFLEN  = 10

10

In [3]:
X    = regular_grid_pts_nD( _X_DOMAIN, [0.25,0.25] );
nPts = size( X )[2]; # ------- Number of anchors
mDim = size( X )[1]; # ------- Dimensionality of anchors 
Y    = zeros(Float64, nPts); # Values at anchors
vsts = zeros(Int64, nPts); # - Set number of visits to zero
println( size(X) )

X_kdTree = KDTree( X; leafsize = _LEAFLEN, reorder = false ); # Prohibit slice reorder

(2, 289)


In [4]:
"""
Compute the Manhattan Distance from `vec` to the origin
"""
function manhattan_norm( vec )
    return sum( abs.( vec ) )
end


"""
Each point contributes the inverse of its manhattan distance from the interpolated point
"""
function manhatt_contrib( vec )
    return 1.0 / manhattan_norm( vec )
end


"""
Calc version of `vec` normalized to sum to 1.0, The contributions of all neighbors must sum to 1.0
"""
function vec_norm( vec )
    len = sum( vec )
    if len == 0.0
        return vec
    else
        return vec / len
    end
end


"""
Return weights for `pnts` according to the closeness to `q`
"""
function manhatt_contrib_to_q( pnts, q )
    Npts  = size( pnts, 2 )
    wghts = zeros( Npts )
    for j = 1:Npts
        wghts[j] = manhatt_contrib( q - pnts[:,j] )
    end
    return vec_norm( wghts )
end

manhatt_contrib_to_q

In [5]:
"""
Use k Nearest Neighbors of `q` to retrive value from a `QStore`
"""
function query_value_fuzzy( kdTree, pnts, vals, q; k = 3 )
    idxs, wgts = K_kNN( kdTree, q; k = k )
    p_  , v_   = fetch_by_indices( pnts, vals, idxs )
    wgts       =  manhatt_contrib_to_q( p_, q )
    return val_weighted_avg( v_, wgts )
end

query_value_fuzzy

In [16]:
truthFunc = get_nD_example_plane_eq( _DIM_X, 1 )
assoc_pnts_with_true_values( X, Y, truthFunc )

In [17]:
bbox = _X_DOMAIN
N    = 50
smpl = sample_bbox_N( bbox, N )
err  = 0.0

for j = 1:N
    smp = smpl[:,j]
    est = query_value_fuzzy( X_kdTree, X, Y, smp; k = 3 )
    tru = truthFunc( smp )
    err += abs( est - tru )
end

println( "Average error for fuzzy kernel on plane: ", err/N )

Average error for fuzzy kernel on plane: 0.02205428957474461


In [18]:
truthFunc = get_nD_example_cone_eq( 4.0, 1.0 )
assoc_pnts_with_true_values( X, Y, truthFunc )

In [19]:
bbox = _X_DOMAIN
N    = 50
smpl = sample_bbox_N( bbox, N )
err  = 0.0

for j = 1:N
    smp = smpl[:,j]
    est = query_value_fuzzy( X_kdTree, X, Y, smp; k = 3 )
    tru = truthFunc( smp )
    err += abs( est - tru )
end

println( "Average error for fuzzy kernel on cone: ", err/N )

Average error for fuzzy kernel on cone: 0.015311012101041323


In [23]:
truthFunc = get_nD_example_hill_eq( 4.0, 2.0 )
assoc_pnts_with_true_values( X, Y, truthFunc )

In [24]:
bbox = _X_DOMAIN
N    = 50
smpl = sample_bbox_N( bbox, N )
err  = 0.0

for j = 1:N
    smp = smpl[:,j]
    est = query_value_fuzzy( X_kdTree, X, Y, smp; k = 3 )
    tru = truthFunc( smp )
    err += abs( est - tru )
end

println( "Average error for fuzzy kernel on hill: ", err/N )

Average error for fuzzy kernel on hill: 0.015776085158719613
